In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re

import pickle 
#import mglearn
import time


from nltk.tokenize import TweetTokenizer # doesn't split at apostrophes
import nltk
from nltk import Text
from nltk.tokenize import regexp_tokenize
from nltk.tokenize import word_tokenize  
from nltk.tokenize import sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

In [2]:
train_df = pd.read_csv("wiki_movie_plots_deduped.csv", delimiter=',')
train_df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [3]:
train_df['Genre length'] = train_df['Genre'].apply(len)
train_df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",7
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",7
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",7
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,7
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,7


In [4]:
train_df['Genre length'].describe()

count    34886.000000
mean         8.700768
std          5.453291
min          1.000000
25%          6.000000
50%          7.000000
75%          9.000000
max         81.000000
Name: Genre length, dtype: float64

In [5]:
train_df.describe()

,Release Year,Genre length
count,34886.000000,34886.000000
mean,1981.314252,8.700768
std,27.815174,5.453291
min,1901.000000,1.000000
25%,1957.000000,6.000000
50%,1988.000000,7.000000
75%,2007.000000,9.000000
max,2017.000000,81.000000


In [6]:
train_df['Genre'].nunique()

2265

There are 2265 type of genres available in the given dataset

In [7]:
genre_arr = train_df['Genre'].unique()

In [8]:
genre_data = train_df['Genre'].value_counts()

In [9]:
genre_data

unknown                                                        6083
drama                                                          5964
comedy                                                         4379
horror                                                         1167
action                                                         1098
                                                               ... 
comedy, social, musical                                           1
horror, thriller, romance, musical                                1
horror, fantasy, action, magical girl, science fantasy, spy       1
biography, horror                                                 1
action, adventure, family, romance                                1
Name: Genre, Length: 2265, dtype: int64

In [10]:
type(genre_data)

pandas.core.series.Series

In [11]:
import string
alpha = string.ascii_lowercase
alpha_up = string.ascii_uppercase
print(alpha)
print(alpha_up)

abcdefghijklmnopqrstuvwxyz
ABCDEFGHIJKLMNOPQRSTUVWXYZ


In [12]:

def remove_punc(data):
    string = ''
    for i in data:
        if i in alpha:
            string = string + i
        elif i in alpha_up:
            string = string + i
        elif i == ' ':
            string = string + i
        else:
            continue
    return string

In [13]:
dat = 'In a mansion called Xanadu, part of a vast palatial estate in Florida, the elderly Charles Foster Kane is on his deathbed. Holding a snow globe, he utters a word, "Rosebud", and dies; the globe slips from his hand and smashes on the floor'
print(remove_punc(dat))

In a mansion called Xanadu part of a vast palatial estate in Florida the elderly Charles Foster Kane is on his deathbed Holding a snow globe he utters a word Rosebud and dies the globe slips from his hand and smashes on the floor


In [16]:
def genre_corr(data):
    data = data.split()
    final_gen = ''
    list_rom = ['romance','love','love story','musical b', 'romantic','rom-coms','music','musical','actionlove','romanceaction',
               'romancecomedy','romancehorror','romcom','rom\|com','rom',' \(artistic\)',"drama|romance|adult|children"
               ]
    list_act = ['act','action','adventures','kung fu','martial arts','world war ii','world war i','spy film','biker film',
               'buddy cop','buddy film','bruceploitation','drama about child soldiers',"war-time","wartime","ww1","wwii",
               'true crime','crime','\|007','gun fu','afghan war drama','actionadventure','actioncomedy','actiondrama',
                'actionlove','actionmasala','actionchildren','adventurecomedy','actionthriller','martialarts',' \(volleyball\)',
                ' \(aquatics|swimming\)',' \(aquatics|swimming\)',' \(shogi|chess\)',' (road bicycle racing)','american football',
                'dev\|nusrat jahan',' \(road bicycle racing\)','liveaction','heistcomedy','heist','historydisaster','warcomedy',
                'samurai','martial_arts','adventure','spy','superhero',"drama|romance|adult|children",'actionner',
               ]
    list_sus = ['ttriller','coming of age','coming-of-age','slice of life','psycho thriller,',"ero",'actionadventure','dramathriller',
                'dramathriller','thriler','crimethriller','actionthriller','comedysocial','erotica','erotic','comedythriller',
                  'colour\|yellow\|productions\|eros\|international',       'melodrama', 'gangsterthriller',  'ancientcostume', 
                'dramatic','biodrama','bio-drama','comedy-drama adaptation of the mordecai richler novel','drama about child soldiers',
                'drama loosely','slice of life',"comedy–drama"'actionlove','actiondrama','fantasycomedy','dramacomedy',
                'dramacomedysocial','dramathriller','comedydrama','comedyhorror','adventurecomedy','animationdrama','comedysocial',
                'erotica','erotic','biblical','biblical','colour\|yellow\|productions\|eros\|international','liveaction','superheroes',
                'heistcomedy','heist','warcomedy','dramatic','familya','familya','dramedy','dramaa','famil\|','superheroe',
                'devotionalbiography','familydrama','espionage','romancefiction','horrorthriller','suspensethriller','triller',
                'satirical','homosexual','sexual','mockumentary','periodic','politics','tv_miniseries','serial',"musical–comedy",
                "roman|porno","action—masala","horror–thriller",'family','martial_arts','horror','war','adventure','noir',
                'superhero','social','suspense',"drama|romance|adult|children",'actionner',
                ]
    list_sci = ['animated','anime','children\'s','3-d','3d','sci-fi','sci fi','science fiction','avant-garde','animationchildren',
               'computer animation',   ' in animation',   'actionchildren',  'fantasychildren\|','fantasycomedy','fantasyperiod',
                'sciencefiction','animationdrama','fantay','\|\(children\|poker\|karuta\)','superheroes','computeranimation',
                '\|\(fiction\)','science_fictionchildren','science_fiction','superhero',"drama|romance|adult|children"
                ]
    list_hor = ['psychological','j-horror','psycho thriller,',"comedy–horror",'actionadventure','comedyhorror','horror',
                ]
    for i in data:
        if i.lower() in list_rom:
            final_gen = final_gen + ' Romance'
        elif i.lower() in list_act:
            final_gen = final_gen + ' Action'
        elif i.lower() in list_sus:
            final_gen = final_gen + ' Suspense'
        elif i.lower() in list_sci:    
            final_gen = final_gen + ' Science Fiction'
        elif i.lower() in list_hor:
            final_gen = final_gen + ' Horror'
        else:
            final_gen = final_gen + ' Others'
    final_gen = set(final_gen.split())
    genre = ''
    for i in final_gen:
        genre = genre + ' {}'.format(i)
    return genre
            

In [18]:
train_df['Genre String'] = train_df['Genre'].apply(remove_punc)

In [20]:
train_df.head(20)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length,Genre String
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",7,unknown
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",7,unknown
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",7,unknown
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,7,unknown
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,7,unknown
5,1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab...",7,unknown
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...,7,western
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...,6,comedy
8,1905,The Little Train Robbery,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Train...,The opening scene shows the interior of the ro...,7,unknown
9,1905,The Night Before Christmas,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Night_Before...,Scenes are introduced using lines of the poem....,7,unknown


In [22]:
train_df['Genre Corrected'] = train_df['Genre String'].apply(genre_corr)

In [23]:
train_df.head(20)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length,Genre String,Genre Corrected
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",7,unknown,Others
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",7,unknown,Others
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",7,unknown,Others
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,7,unknown,Others
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,7,unknown,Others
5,1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab...",7,unknown,Others
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...,7,western,Others
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...,6,comedy,Others
8,1905,The Little Train Robbery,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Train...,The opening scene shows the interior of the ro...,7,unknown,Others
9,1905,The Night Before Christmas,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Night_Before...,Scenes are introduced using lines of the poem....,7,unknown,Others


In [24]:
train_df['Genre Corrected 2'] = train_df['Genre'].apply(genre_corr)

In [25]:
train_df.head(20)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length,Genre String,Genre Corrected,Genre Corrected 2
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",7,unknown,Others,Others
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",7,unknown,Others,Others
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",7,unknown,Others,Others
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,7,unknown,Others,Others
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,7,unknown,Others,Others
5,1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab...",7,unknown,Others,Others
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...,7,western,Others,Others
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...,6,comedy,Others,Others
8,1905,The Little Train Robbery,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Train...,The opening scene shows the interior of the ro...,7,unknown,Others,Others
9,1905,The Night Before Christmas,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Night_Before...,Scenes are introduced using lines of the poem....,7,unknown,Others,Others


In [26]:
train_df['Genre Corrected'].value_counts()

 Others                                    22845
 Action                                     2456
 Suspense                                   2319
 Action Others                              1744
 Others Romance                             1520
 Romance                                    1496
 Suspense Others                            1323
 Science Fiction                             323
 Science Fiction Others                      165
 Suspense Action Others                      102
 Action Romance                              100
 Action Others Romance                        92
 Suspense Others Romance                      91
 Suspense Action                              89
 Suspense Romance                             49
 Fiction Science Others                       34
                                              33
 Others Horror                                28
 Suspense Science Fiction                     18
 Suspense Science Fiction Others              13
 Suspense Action Oth

In [27]:
train_df['Genre Corrected 2'].value_counts()

 Others                                    23794
 Action                                     2314
 Suspense                                   2047
 Others Romance                             1500
 Action Others                              1455
 Romance                                    1433
 Suspense Others                            1278
 Science Fiction                             532
 Science Fiction Others                      186
 Fiction Science Others                      109
 Action Others Romance                        40
 Suspense Action Others                       33
 Others Horror                                31
                                              28
 Suspense Others Romance                      21
 Suspense Romance                             17
 Suspense Action                              16
 Action Romance                               15
 Suspense Horror                               5
 Fiction Science Action Others                 5
 Suspense Science Fi

In [28]:
train_df.drop(['Genre Corrected 2'],axis=1,inplace=True)

In [30]:
train_df.head(20)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length,Genre String,Genre Corrected
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",7,unknown,Others
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",7,unknown,Others
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",7,unknown,Others
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,7,unknown,Others
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,7,unknown,Others
5,1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab...",7,unknown,Others
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...,7,western,Others
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...,6,comedy,Others
8,1905,The Little Train Robbery,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Train...,The opening scene shows the interior of the ro...,7,unknown,Others
9,1905,The Night Before Christmas,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Night_Before...,Scenes are introduced using lines of the poem....,7,unknown,Others


In [31]:
train_df['GenreSplit']=train_df['Genre Corrected'].str.split()

In [33]:
train_df.head(20)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length,Genre String,Genre Corrected,GenreSplit
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",7,unknown,Others,[Others]
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",7,unknown,Others,[Others]
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",7,unknown,Others,[Others]
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,7,unknown,Others,[Others]
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,7,unknown,Others,[Others]
5,1903,Alice in Wonderland,American,Cecil Hepworth,May Clark,unknown,https://en.wikipedia.org/wiki/Alice_in_Wonderl...,"Alice follows a large white rabbit down a ""Rab...",7,unknown,Others,[Others]
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...,7,western,Others,[Others]
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...,6,comedy,Others,[Others]
8,1905,The Little Train Robbery,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Little_Train...,The opening scene shows the interior of the ro...,7,unknown,Others,[Others]
9,1905,The Night Before Christmas,American,Edwin Stanton Porter,NaN,unknown,https://en.wikipedia.org/wiki/The_Night_Before...,Scenes are introduced using lines of the poem....,7,unknown,Others,[Others]


In [39]:
def text_process(word):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    
    # Now just remove any stopwords
    return [i for i in word.split() if i.lower() not in stopwords.words('english')]

In [41]:
for i in range(5):
    print(train_df['Plot'][i])

A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypically Irish man's bucket with beer, Carrie Nation and her followers burst inside. They assault the Irish man, pulling his hat over his eyes and then dumping the beer over his head. The group then begin wrecking the bar, smashing the fixtures, mirrors, and breaking the cash register. The bartender then sprays seltzer water in Nation's face before a group of policemen appear and order everybody to leave.[1]
The moon, painted with a smiling face hangs over a park at night. A young couple walking past a fence learn on a railing and look up. The moon smiles. They embrace, and the moon's smile gets bigger. They then sit down on a bench by a tree. The moon's view is blocked, causing him to frown. In the last scene, the man fans the woman with his hat because the moon has left the sky and is perched over her shoulder to see everything better.
The film, just over a minute long, is composed of two shots. 

In [44]:
train_df['Plot without punc'] = train_df['Plot'].apply(remove_punc)

In [50]:
train_df2 = train_df.iloc[12500:15000]
train_df2

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,Genre length,Genre String,Genre Corrected,GenreSplit,Plot without punc
12500,1995,Miami Rhapsody,American,David Frankel,"Sarah Jessica Parker, Antonio Banderas, Mia Fa...",comedy,https://en.wikipedia.org/wiki/Miami_Rhapsody,Gwyn Marcus (Sarah Jessica Parker) has always ...,6,comedy,Others,[Others],Gwyn Marcus Sarah Jessica Parker has always wa...
12501,1995,A Midwinter's Tale,American,Kenneth Branagh,Michael Maloney,drama,https://en.wikipedia.org/wiki/A_Midwinter%27s_...,"The story is set in Hope, Derbyshire, England....",5,drama,Others,[Others],The story is set in Hope Derbyshire England It...
12502,1995,Mighty Aphrodite,American,Woody Allen,"Woody Allen, Mira Sorvino, Helena Bonham Carte...",comedy,https://en.wikipedia.org/wiki/Mighty_Aphrodite,The film opens on ancient Greek ruins where a ...,6,comedy,Others,[Others],The film opens on ancient Greek ruins where a ...
12503,1995,Mighty Morphin Power Rangers: The Movie,American,Bryan Spicer,"Karan Ashley, Johnny Yong Bosch, Steve Cardena...","science fiction, family",https://en.wikipedia.org/wiki/Mighty_Morphin_P...,The Power Rangers participate with Bulk and Sk...,23,science fiction family,Suspense Others,"[Suspense, Others]",The Power Rangers participate with Bulk and Sk...
12504,1995,Money Train,American,Joseph Ruben,"Wesley Snipes, Woody Harrelson, Jennifer Lopez...",action,https://en.wikipedia.org/wiki/Money_Train,"On Christmas, we meet foster brothers John and...",6,action,Action,[Action],On Christmas we meet foster brothers John and ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2006,Mad Cowgirl,American,Gregory Hatanaka,"Sarah Lassez, James Duval, Walter Koenig",drama,https://en.wikipedia.org/wiki/Mad_Cowgirl,The central character in Mad Cowgirl is Theres...,5,drama,Others,[Others],The central character in Mad Cowgirl is Theres...
14996,2006,Madea's Family Reunion,American,Tyler Perry,"Tyler Perry, Blair Underwood, Lynn Whitfield, ...",comedy-drama,https://en.wikipedia.org/wiki/Madea%27s_Family...,After Madea (Tyler Perry) violates the terms o...,12,comedydrama,Suspense,[Suspense],After Madea Tyler Perry violates the terms of ...
14997,2006,Man About Town,American,Mike Binder,"Ben Affleck, Rebecca Romijn, John Cleese, Jerr...",comedy,https://en.wikipedia.org/wiki/Man_About_Town_(...,"Top Hollywood talent agent, Jack Giamoro (Ben ...",6,comedy,Others,[Others],Top Hollywood talent agent Jack Giamoro Ben Af...
14998,2006,Man of the Year,American,Barry Levinson,"Robin Williams, Laura Linney",comedy,https://en.wikipedia.org/wiki/Man_of_the_Year_...,"Tom Dobbs is host of a satirical news program,...",6,comedy,Others,[Others],Tom Dobbs is host of a satirical news program ...


In [ ]:
train_df2['Plot final clean'] = train_df2['Plot without punc'].apply(text_process)